In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
from skimage.io import imread
import numpy as np

from prepare_lyft_data import extract_single_box, \
    parse_train_csv, level5data, extract_boxed_clouds, \
    get_sample_images, get_train_data_sample_token_and_box, \
    get_pc_in_image_fov, get_bounding_box_corners, \
    get_2d_corners_from_projected_box_coordinates, transform_image_to_world_coordinate,\
transform_world_to_image_coordinate
from lyft_dataset_sdk.utils.data_classes import LidarPointCloud,Quaternion

In [3]:
train_df = parse_train_csv()
sample_token, bounding_box = get_train_data_sample_token_and_box(0, train_df)
first_train_sample = level5data.get('sample', sample_token)



100%|██████████| 22680/22680 [00:03<00:00, 7112.88it/s] 


In [4]:
cam_token = first_train_sample['data']['CAM_FRONT']

box_corners = get_bounding_box_corners(bounding_box, cam_token)

# check)image
cam_image_file = level5data.get_sample_data_path(cam_token)
cam_image_mtx = imread(cam_image_file)

xmin, xmax, ymin, ymax = get_2d_corners_from_projected_box_coordinates(box_corners)

random_depth = 20
image_center = np.array([[(xmax + xmin) / 2, (ymax + ymin) / 2, random_depth]]).T

In [5]:
image_wc = transform_image_to_world_coordinate(image_center, cam_token)

[[-16.89750522]
 [ -9.19058195]
 [ 20.        ]]
{'sensor_token': 'eb9e8f60a3d6e3328d7512b9f8e6800127fe91f4d62bc8e48a0e6a7cb116cc60', 'rotation': [0.5090416344726354, -0.49422295323980653, 0.4930125688749941, -0.5035463174283287], 'camera_intrinsic': [[1109.05239567, 0, 957.849065461], [0.0, 1109.05239567, 539.672710373], [0.0, 0.0, 1.0]], 'translation': [1.5039405282244198, -0.02676183592864872, 1.6584901808053665], 'token': '8e73e320d1fa9e5af96059e6eb1dd7d28e3271dea04de86ead47fa25fd13fd20'}
in pose record: [[-15.39356469]
 [ -9.21734378]
 [ 21.65849018]]


In [6]:
image_wc

array([[2.72451762e+03],
       [6.94520535e+02],
       [2.29332199e+00]])

### confirm that the projection is correct

In [7]:
transform_world_to_image_coordinate(image_wc,cam_token)

array([[313.60157197],
       [583.05679071],
       [  1.        ]])

In [8]:
image_center

array([[416.76266258],
       [600.61728081],
       [ 20.        ]])

## Dissect the steps

In [55]:
def normalization(input_array):
    input_array[0:2,:]=input_array[0:2,:]*input_array[2:3,:].repeat(2,0).reshape(2,input_array.shape[1])
    return input_array

In [56]:
image_array=np.copy(image_center)
camera_token=cam_token

In [57]:
image_array=normalization(image_array)
image_array=np.concatenate((image_array.ravel(),np.array([1])))
image_array=image_array.reshape(4,1)

In [58]:
image_array

array([[8.33525325e+03],
       [1.20123456e+04],
       [2.00000000e+01],
       [1.00000000e+00]])

In [59]:
sd_record = level5data.get("sample_data", camera_token)
cs_record = level5data.get("calibrated_sensor", sd_record["calibrated_sensor_token"])
sensor_record = level5data.get("sensor", cs_record["sensor_token"])
pose_record = level5data.get("ego_pose", sd_record["ego_pose_token"])

# inverse the viewpoint transformation


cam_intrinsic_mtx = np.array(cs_record["camera_intrinsic"])
view=cam_intrinsic_mtx
viewpad=np.eye(4)
viewpad[: view.shape[0], : view.shape[1]] = view
image_in_cam_coord = np.dot(np.linalg.inv(viewpad), image_array)

print("image_in_cam_coord",image_in_cam_coord)
# TODO: think of how to do normalization properly
# image_in_cam_coord = image_in_cam_coord / image_in_cam_coord[3:].ravel()

# homogeneous coordinate to non-homogeneous one
image_in_cam_coord = image_in_cam_coord[0:3, :]

sens_to_pose_rot_mtx = Quaternion(cs_record['rotation']).rotation_matrix

image_in_pose_coord = np.dot(sens_to_pose_rot_mtx, image_in_cam_coord)
t = np.array(cs_record['translation'])
for i in range(3):
    image_in_pose_coord[i, :] = image_in_cam_coord[i, :] + t[i]
    

print("in pose record:", image_in_pose_coord)

pose_to_world_rot_mtx = Quaternion(pose_record['rotation']).rotation_matrix

image_in_world_coord = np.dot(pose_to_world_rot_mtx,
                              image_in_pose_coord)
t = np.array(pose_record['translation'])
for i in range(3):
    image_in_world_coord[i, :] = image_in_world_coord[i, :] + t[i]

image_in_cam_coord [[-9.75763463]
 [ 1.09903861]
 [20.        ]
 [ 1.        ]]
in pose record: [[-8.2536941 ]
 [ 1.07227678]
 [21.65849018]]


In [60]:
image_in_world_coord

array([[2.71298418e+03],
       [6.89641610e+02],
       [2.46603677e+00]])

In [61]:
transform_world_to_image_coordinate(image_in_world_coord,cam_token)

array([[315.046435  ],
       [583.07015896],
       [  1.        ]])

In [62]:
image_in_cam_coord[2:3,:].repeat(3,0).reshape(3,image_array.shape[1])

array([[20.],
       [20.],
       [20.]])

In [63]:
def project_image_to_rect(uv_depth):
    ''' Input: nx3 first two channels are uv, 3rd channel
               is depth in rect camera coord.
        Output: nx3 points in rect camera coord.
    '''
    
    n = uv_depth.shape[0]
    x = ((uv_depth[:,0]-c_u)*uv_depth[:,2])/f_u + b_x
    y = ((uv_depth[:,1]-c_v)*uv_depth[:,2])/f_v + b_y
    pts_3d_rect = np.zeros((n,3))
    pts_3d_rect[:,0] = x
    pts_3d_rect[:,1] = y
    pts_3d_rect[:,2] = uv_depth[:,2]
    return pts_3d_rect

## Only to camera coordinates

In [65]:
cam_intrinsic_mtx = np.array(cs_record["camera_intrinsic"])
view=cam_intrinsic_mtx
viewpad=np.eye(4)
viewpad[: view.shape[0], : view.shape[1]] = view
image_in_cam_coord = np.dot(np.linalg.inv(viewpad), image_array)

print("image_in_cam_coord",image_in_cam_coord)

image_in_cam_coord [[-9.75763463]
 [ 1.09903861]
 [20.        ]
 [ 1.        ]]


In [66]:
from lyft_dataset_sdk.utils.geometry_utils import view_points

In [69]:
view_points(image_in_cam_coord[0:3,:],view=cam_intrinsic_mtx,normalize=True)

array([[416.76266258],
       [600.61728081],
       [  1.        ]])